In [ ]:
##We load the packages used at the work
##We are going to use as rule pd for Pandas
##If you are going to use Matplotlib, Numpy or
##other, you should search how to name them 
## as rule.
##For keep the order you can use prettier
## in vscode


##Pandas is similar to tidyverse
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
##The next package is used to format tables
##In the next lines you can watch how to use it
from tabulate import tabulate
from sklearn import svm
from sklearn import metrics

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import RocCurveDisplay
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split


from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

##La tabla que vamos a emplear es esta, las demás han
##sido eliminadas
file="SouthGermanCredit.asc"
data=pd.read_csv(file," ")

##We are going to change the name of the variables 
##because in German is difficult to us work.
names={'laufkont':'Status',
        'laufzeit':'Duracion',
        'moral':'Historial_Crediticio',
        'verw':'Proposito',
        'hoehe':'Monto',
        'sparkont':'Ahorros',
        'beszeit':'Tiempo_trabajando',
        'rate':'Porcentaje_salario',
        'famges':'Estado_civil_sexo',
        'buerge':'Otros_deudores',
        'wohnzeit':'Residencia',
        'verm':'Propiedad_valiosa',
        'alter':'Edad',
        'weitkred':'Otras_deudas',
        'wohn':'Residencia',
        'bishkred':'Creditos_banco',
        'beruf':'Empleo',
        'pers':'Personas_dependientes',
        'telef':'Telefono',
        'gastarb':'Extranjero',
        'kredit':'Default',}


data.rename(columns=names,inplace=True)
#print(data.head(5))
#print(tabulate(data.head(5),headers='keys',tablefmt='latex_longtable'))


In [ ]:
##It's contructed a table of min and max of all to know if 
##the data doesn't have mistakes
data_1=data.agg(['min','max'])
print(tabulate(data_1.transpose(),headers='keys',tablefmt='latex_longtable'))

In [ ]:
##We are going to construct the tables
##We are going to do a resume of five numbers
##Duracion, Monto, Edad
##First we select this three columns
##This new variable is the data.frame with,
##what we want
data_2=data[['Duracion','Edad','Monto','Tiempo_trabajando']]

summary_2=data_2.describe().transpose()
summary_2=summary_2.loc[:,summary_2.columns!='count']



print(tabulate(summary_2,headers='keys',tablefmt='latex_longtable'))

In [ ]:
#We estimate the quartile coefficient of dispersion
#summary_2['Coeficiente de dispersion cuartil']=(summary_2['75%']-summary_2['25%'])/(summary_2['75%']+summary_2['25%'])
#summary_2['Coeficiente de variacion']=summary_2['std']/summary_2['mean']

#print(tabulate(summary_2[['Coeficiente de dispersion cuartial','Coeficiente de variacion']],headers='keys',floatfmt=".4f",tablefmt='latex_longtable'))

In [ ]:
##A table count historial and default
historial_default=pd.crosstab(index=data['Default'],columns=data['Historial_Crediticio'],margins=True)
print(tabulate(historial_default,headers='keys',tablefmt='latex_longtable'))

In [ ]:
##A table count savings and default
savings_default=pd.crosstab(index=data['Default'],columns=data['Ahorros'])
print(tabulate(savings_default,headers='keys',tablefmt='latex_longtable'))

In [ ]:
#Before we start we are going to change the model form 0 to -1
data.loc[data.Default==0,'Default']=-1

#Here we are going to construct the way in wich we construct the sample
#Neverthless what we want to make to this is a model that at this moment have not been programmed so we are going to select the sample dividing the table in two the good and the bads 
#In this case we are going to select 80% of the good and bads, and also for
#20% of training
goods=data[(data['Default']==1)]
bads=data[(data['Default']==-1)]
#With this we construct the sample

goods_random=goods.sample(n=round(len(goods)*0.8),replace=False)
bads_random=bads.sample(n=round(len(bads)*0.8),replace=False)
train=pd.concat([goods_random,bads_random])
test_random=data.drop(train.index)
X_train=train.drop('Default',1).values.tolist()
Y_train=train[['Default']]
X_test=test_random.drop('Default',1).values.tolist()
Y_test_real=test_random[['Default']]


In [ ]:
#Here we are going to program the svm, but with the four basic kernels,
#and with the same gamma
#We are going to keep the basic adjustment for in this case
#And we compare also it with the proposed by Ben-David and Shalev

##Linear, This is not accomplised for our data set
#linear_svm=svm.SVC(kernel='linear',probability=True)
#linear_svm.fit(X_train,Y_train)

##Polynomial of grade 2
poly_svm=svm.SVC(kernel='poly',degree=2,probability=True)
poly_svm.fit(X_train,Y_train)



##Gaussian Kernel
gauss_svm=svm.SVC(kernel='rbf',probability=True)
gauss_svm.fit(X_train,Y_train)

##sigmoid
sigm_svm=svm.SVC(kernel='sigmoid',probability=True)
sigm_svm.fit(X_train,Y_train)


In [ ]:
# We are going to construct the model here based in the model
#presented by Ben-David and Shalev
##Polynomial of grade 2
poly_svm_bd=svm.SVC(kernel='poly',degree=2,probability=True,C=1/len(test_random))
poly_svm_bd.fit(X_train,Y_train)



##Gaussian Kernel
gauss_svm_bd=svm.SVC(kernel='rbf',probability=True,C=1/len(test_random))
gauss_svm_bd.fit(X_train,Y_train)

##sigmoid
sigm_svm_bd=svm.SVC(kernel='sigmoid',probability=True,C=1/len(test_random))
sigm_svm_bd.fit(X_train,Y_train)



In [ ]:
#confusion matrix

def confusion(modelo):
    Predicciones =modelo.predict(test_random.drop('Default',1))
    fir_matrix=confusion_matrix(test_random.loc[:,'Default'], Predicciones, labels=[1,-1])
    matrix_conf=ConfusionMatrixDisplay(fir_matrix, display_labels=['1','0'])
    matrix_conf.plot(colorbar=False)
    matrix_conf.ax_.set(xlabel='Predicciones', ylabel='Datos verdaderos')

In [ ]:
#ROC and AUC
def curvasROC(modelo):
    Predicciones =modelo.predict(test_random.drop('Default',1))
    tn, fp, fn, tp = confusion_matrix(test_random.loc[:,'Default'],Predicciones).ravel()
    tpr=tp/(tp+fn)
    fpr=fp/(fp+tn)
    roc_auc = metrics.auc(([0., fpr, 1.]),([0., tpr, 1.]))

    fig, ax = plt.subplots()
    Probabilistico=RocCurveDisplay.from_estimator(modelo, test_random.drop('Default',1),
     test_random.loc[:,'Default'], color= "#FF6103" , ax = ax, name = "ROC probabilística")

    display=RocCurveDisplay(fpr=([0., fpr, 1.]), tpr=([0., tpr, 1.]),  roc_auc=roc_auc,
                                   estimator_name='ROC puntual')
    display.plot(color="#03A89E",ax=ax)

    plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")  
    plt.scatter(fpr, tpr, color ="#03A89E" )
    plt.xlabel("1-Especificidad")
    plt.ylabel("Sencibilidad")
    plt.title("ROC")

In [ ]:
confusion(gauss_svm)
curvasROC(gauss_svm)